In [1]:
import requests 
import json
import time
import os
import pandas as pd
from sqlalchemy import engine as sql
from IPython import display
import ipywidgets as widget
import matplotlib.pyplot as plt
import math

In [8]:
data = requests.get('https://api.hh.ru/vacancies?text=java').json()
data

{'items': [{'id': '44706219',
   'premium': False,
   'name': 'Junior Java Developer',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
   'salary': {'from': 90000, 'to': 140000, 'currency': 'RUR', 'gross': False},
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': None,
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2021-06-12T11:39:02+0300',
   'created_at': '2021-06-12T11:39:02+0300',
   'archived': False,
   'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=44706219',
   'insider_interview': None,
   'url': 'https://api.hh.ru/vacancies/44706219?host=hh.ru',
   'alternate_url': 'https://hh.ru/vacancy/44706219',
   'relations': [],
   'employer': {'id': '193245',
    'name': 'INTECH',
    'url': 'https://api.hh.ru/employers/193245',
    'alternate_url': 'https://hh.ru/employer/193245',
    'logo_urls': {'origi

# Собираем данные по специальности

In [2]:

 
def getPage(page = 0):
    params = {
        'text': 'NAME:Data Scientist', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data
 
for page in range(0, 20):
    jsObj = json.loads(getPage(page))
    nextFileName = './docs/pagination/{}.json'.format(len(os.listdir('./docs/pagination')))
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    if (jsObj['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
    time.sleep(0.25)
     
print('Старницы поиска собраны')

Старницы поиска собраны


# Собираем данные по вакансии

In [3]:
 
# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
for fl in os.listdir('./docs/pagination'):
     
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open('./docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)
     
    # Получаем и проходимся по непосредственно списку вакансий
    for v in jsonObj['items']:
         
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
         
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = './docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
         
        time.sleep(0.25)
         
print('Вакансии собраны')

Вакансии собраны


# Собираем данные в  базу данных

In [59]:
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
 
# Создаем списки для столбцов таблицы skills
skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список названий навыков
 
# В выводе будем отображать прогресс
# Для этого узнаем общее количество файлов, которые надо обработать
# Счетчик обработанных файлов установим в ноль
cnt_docs = len(os.listdir('./docs/vacancies'))
i = 0
 
# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./docs/vacancies'):
     
    # Открываем, читаем и закрываем файл
    f = open('./docs/vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
     
    # Заполняем списки для таблиц
    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])
    descriptions.append(jsonObj['description'])
     
    # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
    for skl in jsonObj['key_skills']:
        skills_vac.append(jsonObj['id'])
        skills_name.append(skl['name'])
     
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    display.clear_output(wait=True)
    display.display('Готово {} из {}'.format(i, cnt_docs))
 
 

# Создаем пандосовский датафрейм, который затем сохраняем в БД в таблицу vacancies
df1 = pd.DataFrame({'id': IDs, 'name': names, 'description': descriptions})
 
# Тоже самое, но для таблицы skills
df2 = pd.DataFrame({'vacancy': skills_vac, 'skill': skills_name})

 
# Выводим сообщение об окончании программы
display.clear_output(wait=True)
display.display('Вакансии загружены в БД')

'Вакансии загружены в БД'

# Top skills

In [60]:
df2.rename(columns={'vacancy': 'id'}, inplace=True)

In [61]:
dfs=df2['skill'].value_counts()
dfs.to_frame().reset_index()[:11]

,index,skill
0,SQL,822
1,Python,446
2,Бизнес-анализ,427
3,MS PowerPoint,322
4,MS SQL,275
5,Аналитическое мышление,262
6,Анализ данных,243
7,Аналитические исследования,241
8,Английский язык,234
9,MS Visio,214


In [55]:
dfs

SQL                               822
Python                            446
Бизнес-анализ                     427
MS PowerPoint                     322
MS SQL                            275
                                 ... 
Продажа страховых продуктов         1
azure                               1
Организация учебного процесса       1
Моделирование бизнес-процессов      1
Расчет заработной платы             1
Name: skill, Length: 1273, dtype: int64

# Some work with data and expirements 

In [12]:
df2.rename(columns={'vacancy': 'id'}, inplace=True)
df2.merge(df1)

,id,skill
0,33368305,VBA
1,33368305,QlickView
2,33368305,Olap (online analytical processing)
3,36880234,Управление проектами
4,36880234,Бизнес-анализ
...,...,...
11183,45421621,UML
11184,45421621,BPMN
11185,45421621,СУБД Oracle
11186,45421621,MS SQL


In [15]:
 df2=df2.merge(df1)
df2

,id,skill,name,description
0,33368305,VBA,Аналитик - математик (отдел закупа),<p><em>В связи с развитием сети и структуры от...
1,33368305,QlickView,Аналитик - математик (отдел закупа),<p><em>В связи с развитием сети и структуры от...
2,33368305,Olap (online analytical processing),Аналитик - математик (отдел закупа),<p><em>В связи с развитием сети и структуры от...
3,36880234,Управление проектами,Аналитик,<p><em><strong>Lenvendo </strong>на протяжении...
4,36880234,Бизнес-анализ,Аналитик,<p><em><strong>Lenvendo </strong>на протяжении...
...,...,...,...,...
11183,45421621,UML,Аналитик,"<strong>Обязанности:</strong> <ul> <li>Сбор, с..."
11184,45421621,BPMN,Аналитик,"<strong>Обязанности:</strong> <ul> <li>Сбор, с..."
11185,45421621,СУБД Oracle,Аналитик,"<strong>Обязанности:</strong> <ul> <li>Сбор, с..."
11186,45421621,MS SQL,Аналитик,"<strong>Обязанности:</strong> <ul> <li>Сбор, с..."


In [17]:
 res = df2.groupby('id')['skill'].apply(list)
type(res)

pandas.core.series.Series

In [33]:
res=df2.groupby('id')['skill'].apply(list)
res

id
33368305    [VBA, QlickView, Olap (online analytical proce...
36880234    [Управление проектами, Бизнес-анализ, MS Proje...
37510377    [SQL, Python, Бизнес-анализ, Tableau, Разработ...
37625294                                        [Python, SQL]
37854983    [Информационная безопасность, Поиск уязвимосте...
                                  ...                        
45418312    [Python, Data Mining, SQL, MS Visio, Machine L...
45418322    [Опыт работы в автомобильном бизнесе от 2 лет....
45419562                          [XML, SQL, UML, BPMN, REST]
45420741    [SQL, Olap (online analytical processing), MS ...
45421621    [Разработка технических заданий, Бизнес-анализ...
Name: skill, Length: 1811, dtype: object

In [35]:
res=res.to_frame().reset_index()
res=res.merge(df1)

In [36]:
res.pop('description')

0       <p><em>В связи с развитием сети и структуры от...
1       <p><em><strong>Lenvendo </strong>на протяжении...
2       Мы ищем бизнес-аналитика BI, который будет уча...
3       <p><strong>Обязанности:</strong></p> <p>Разраб...
4       <p>Дочерняя компания АО «ИнфоТеКС» «Перспектив...
                              ...                        
1806    <p><strong>Обязанности:</strong></p> <ul> <li>...
1807    <strong>Обязанности:</strong> <ul> <li>Анализи...
1808    <p>Требования:</p> <ul> <li>Сбор, формализация...
1809    <p>Компания «Премьер аналитика» оказывает услу...
1810    <strong>Обязанности:</strong> <ul> <li>Сбор, с...
Name: description, Length: 1811, dtype: object

In [37]:
res

,id,skill,name
0,33368305,"[VBA, QlickView, Olap (online analytical proce...",Аналитик - математик (отдел закупа)
1,36880234,"[Управление проектами, Бизнес-анализ, MS Proje...",Аналитик
2,37510377,"[SQL, Python, Бизнес-анализ, Tableau, Разработ...",Бизнес - аналитик BI
3,37625294,"[Python, SQL]",Data scientist в управление розничного моделир...
4,37854983,"[Информационная безопасность, Поиск уязвимосте...",Аналитик pentest
...,...,...,...
1806,45418312,"[Python, Data Mining, SQL, MS Visio, Machine L...",Senior Data Scientist
1807,45418322,[Опыт работы в автомобильном бизнесе от 2 лет....,Аналитик контента
1808,45419562,"[XML, SQL, UML, BPMN, REST]",Ведущий Аналитик
1809,45420741,"[SQL, Olap (online analytical processing), MS ...",Аналитик (специалист по планированию)
